In [36]:
# Requirements:
# !pip install rtdl
# !pip install libzero==0.0.4

In [1]:
from typing import Any, Dict

import numpy as np
import rtdl
import scipy.special
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
import zero
import pandas as pd

In [2]:
device = torch.device('cpu')
# Docs: https://yura52.github.io/zero/0.0.4/reference/api/zero.improve_reproducibility.html
# zero.improve_reproducibility(seed=123456)

### Data

In [7]:
from sklearn.datasets import fetch_kddcup99 
data = fetch_kddcup99()

from pprint import pprint
print(list(data.target_names))

['labels']


In [8]:
df = pd.DataFrame(data.data, columns=data.feature_names)
df["labels"] = data.target
df.to_csv("data/KDD99/fetch_kddcup99.csv", index=False)

In [36]:
path = "data/Forest_Cover/train_centered.csv"

target = "Cover_Type"
df = pd.read_csv(path)

In [17]:
df["target"] = df[target]# df.income
df = df.drop(target, axis=1)

KeyError: 'labels'

In [41]:
strCol = []
for column in df.columns:
    print(column)
    if type(df[column][0]) is str:
        strCol.append(column)


duration
protocol_type
service
flag
src_bytes
dst_bytes
land
wrong_fragment
urgent
hot
num_failed_logins
logged_in
num_compromised
root_shell
su_attempted
num_root
num_file_creations
num_shells
num_access_files
num_outbound_cmds
is_host_login
is_guest_login
count
srv_count
serror_rate
srv_serror_rate
rerror_rate
srv_rerror_rate
same_srv_rate
diff_srv_rate
srv_diff_host_rate
dst_host_count
dst_host_srv_count
dst_host_same_srv_rate
dst_host_diff_srv_rate
dst_host_same_src_port_rate
dst_host_srv_diff_host_rate
dst_host_serror_rate
dst_host_srv_serror_rate
dst_host_rerror_rate
dst_host_srv_rerror_rate
target


In [42]:
strCol = ["Soil_Type", "Wilderness_Area"]

strCol = ["protocol_type", "service", "flag", "land"]



In [43]:
# df = pd.DataFrame(data.data.tolist(), columns=data.feature_names)
oldNames = df.columns# data.feature_names

In [44]:
# df["target"] = data.target
# df["target"] = df.income

In [45]:
output = df.target.values
labels = set(output)
print('The different type of output labels are:',labels)
print('='*125)
print('No. of different output labels are:', len(labels))

The different type of output labels are: {b'multihop.', b'buffer_overflow.', b'teardrop.', b'ipsweep.', b'satan.', b'pod.', b'portsweep.', b'loadmodule.', b'nmap.', b'back.', b'imap.', b'perl.', b'normal.', b'neptune.', b'guess_passwd.', b'warezmaster.', b'smurf.', b'rootkit.', b'land.', b'phf.', b'warezclient.', b'ftp_write.', b'spy.'}
No. of different output labels are: 23


In [46]:
def onhotencode(_df, _col):
    _values = set(_df[_col].values)
    for v in _values:
        _df[v] = _df[_col].apply(lambda x : float(x == v) )
    return _df

In [47]:
for c in df.columns:
    if (not c in strCol) and (c != "target"):
        df = df.drop(c, axis=1)

In [48]:
for col in df.columns:
    if col != "target":
        df = onhotencode(df, col)
        df = df.drop(col, axis=1)

In [49]:
df["target"] = df["target"].apply(lambda x: str(x))

In [52]:
df.to_csv("data/KDD99/training_processed.csv", index=False)

In [30]:
# df.to_csv("data/Forest_Cover/training_processed.csv")

In [56]:
set(list(df.target))

{"b'back.'",
 "b'buffer_overflow.'",
 "b'ftp_write.'",
 "b'guess_passwd.'",
 "b'imap.'",
 "b'ipsweep.'",
 "b'land.'",
 "b'loadmodule.'",
 "b'multihop.'",
 "b'neptune.'",
 "b'nmap.'",
 "b'normal.'",
 "b'perl.'",
 "b'phf.'",
 "b'pod.'",
 "b'portsweep.'",
 "b'rootkit.'",
 "b'satan.'",
 "b'smurf.'",
 "b'spy.'",
 "b'teardrop.'",
 "b'warezclient.'",
 "b'warezmaster.'"}